In [2]:
!pip install gym
!pip install pygame

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 9.0 MB/s eta 0:00:0000:0100:01


In [3]:
import gym
import numpy as np

In [4]:
env = gym.make('CartPole-v1', render_mode='human')

/Users/martina/opt/anaconda3/envs/gym/lib/python3.10/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/Users/martina/opt/anaconda3/envs/gym/lib/python3.10/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [5]:
def epsilon_greedy_policy(state, Q, epsilon=0.1):
    explore = np.random.binomial(1, epsilon)
    if explore:
        action = env.action_space.sample()
        print('explore')
    else:
        action = np.argmax(Q[state])
        print('exploit')
    return action

In [6]:
def optimal_policy(state, Q):
    action = np.argmax(Q[state])
    return action

In [12]:
bins_cart_position = np.logspace(-4.8, 4.8, 2)
bins_cart_velocity = np.linspace(-1, 1, 2)
bins_pole_angle = np.logspace(-0.2095, 0.2095, 2)
bins_pole_angle_velocity = np.linspace(-1,1,2)

In [25]:
def get_state(obs):
    state_bins_cart_position = np.digitize(obs[0], bins_cart_position)
    state_bins_cart_velocity= np.digitize(obs[1], bins_cart_velocity)
    state_bins_pole_angle = np.digitize(obs[2], bins_pole_angle)
    state_pole_angle_velocity = np.digitize(obs[3], bins_pole_angle_velocity)

    state = state_bins_cart_position, state_bins_cart_velocity, state_bins_pole_angle,state_pole_angle_velocity
    return state

In [26]:
state = get_state([-1.4, -2., 0.23, 1.2])
state

(0, 0, 0, 2)

In [19]:
Q = np.random.random((2,2,2,2,2))
print(Q)

[[[[[0.30926953 0.37026238]
    [0.57332174 0.05783227]]

   [[0.56560201 0.31495698]
    [0.8702623  0.41045653]]]


  [[[0.75721603 0.77821482]
    [0.19135944 0.8744862 ]]

   [[0.62961487 0.04792157]
    [0.45118673 0.78435251]]]]



 [[[[0.96068397 0.17646333]
    [0.47832725 0.04306344]]

   [[0.83661755 0.77301267]
    [0.11736368 0.36846329]]]


  [[[0.82540091 0.12965178]
    [0.21317468 0.90958554]]

   [[0.51421961 0.80664246]
    [0.15616299 0.57937222]]]]]


In [27]:
def updateQ( Q, state, action, reward, newState, alfa , gamma):
    Q[state][action] = Q[state][action] + alfa*(reward + gamma*(np.max(Q[newState])) - Q[state][action])

In [40]:
def play(alfa, gamma, epsilon, Q):
    obs =env.reset()
    #print(obs)
    done = False
    while not done:
        state = get_state(obs)
        action = epsilon_greedy_policy(state, Q, 0.5)
        obs, reward, done, info = env.step(action)
        updateQ(Q, state, action, reward, get_state(obs), alfa, gamma)
        print(Q, state, action, reward, get_state(obs), alfa, gamma)
        #print('->', state, action, reward, obs, done, info)
    return Q
# env.close()

In [41]:
alfa = 0.9
gamma = 0.8
epsilon = 0.9

play(alfa, gamma, epsilon, Q)




exploit
[[[[[0.30926953 0.37026238]
    [0.57332174 0.05783227]]

   [[0.56560201 0.31495698]
    [0.8702623  0.41045653]]]


  [[[0.75721603 0.77821482]
    [0.19135944 1.73284704]]

   [[0.62961487 0.04792157]
    [0.45118673 0.78435251]]]]



 [[[[0.96068397 0.17646333]
    [0.47832725 0.04306344]]

   [[0.83661755 0.77301267]
    [0.11736368 0.36846329]]]


  [[[0.82540091 0.12965178]
    [0.21317468 1.64586014]]

   [[0.51421961 0.80664246]
    [0.15616299 0.57937222]]]]] (1, 1, 0, 1) 1 1.0 (1, 1, 0, 1) 0.9 0.8
-> (1, 1, 0, 1) 1 1.0 [ 0.0471743   0.19719216 -0.00090343 -0.2831617 ] False {}
explore
[[[[[0.30926953 0.37026238]
    [0.57332174 0.05783227]]

   [[0.56560201 0.31495698]
    [0.8702623  0.41045653]]]


  [[[0.75721603 0.77821482]
    [0.19135944 1.73284704]]

   [[0.62961487 0.04792157]
    [0.45118673 0.78435251]]]]



 [[[[0.96068397 0.17646333]
    [0.47832725 0.04306344]]

   [[0.83661755 0.77301267]
    [0.11736368 0.36846329]]]


  [[[0.82540091 0.12965178]
    [

IndexError: index 2 is out of bounds for axis 3 with size 2